 # Building A Web Browsing Agent to Manage my Letterboxd Account

 Let's build a cool simple agent that can perform all sorts of tasks related to movie and cinema in the letterboxd website.

The tools of this agent will be:

- Searching movies
- Searching movie lists
- Add movies to my watch list
- Removing movies from my watchlist
- Marking when I've already seen a movie from my watchlist
- Writing movie reviews based on my impressions

For each of these actions we'll define a tool just like we've worked on before and from there we'll setup our agent with different capabilities and test it. 

Let's start with the login to the account.

```python

```

Now, let's create specified tools.